In [ ]:
!pip install allensdk

In [ ]:
from allensdk.core.cell_types_cache import CellTypesCache
from allensdk.ephys.extract_cell_features import extract_cell_features
from allensdk.ephys.extract_cell_features import extract_sweep_features
from collections import defaultdict
from allensdk.core.cell_types_cache import CellTypesCache
from allensdk.api.queries.cell_types_api import CellTypesApi
from allensdk.core.cell_types_cache import ReporterStatus as RS
# Instantiate the CellTypesCache instance.  The manifest_file argument
# tells it where to store the manifest, which is a JSON file that tracks
# file paths.  If you supply a relative path it will go into your
# current working directory
ctc = CellTypesCache()
from allensdk.core.nwb_data_set import NwbDataSet
import pickle 



In [ ]:
cells = ctc.get_cells(species=[CellTypesApi.HUMAN])
# download all cells(cache)
# get sequence list
def get_sequenceList():
    cells = ctc.get_cells()
#     print("Total cells: %d" % len(cells))
    specimen_id_list=[]
    for i in range(0, len(cells)):
        specimen_id_list.append(cells[i]['id'])
    return specimen_id_list

specimen_id_list=get_sequenceList()

In [ ]:
 
# download all cells(cache)
# get sequence list
def get_human_sequenceList():
    cells = ctc.get_cells(species=[CellTypesApi.HUMAN])
#     print("Total cells: %d" % len(cells))
    specimen_id_list=[]
    for i in range(0, len(cells)):
        specimen_id_list.append(cells[i]['id'])
    return specimen_id_list

specimen_id_human_list=get_human_sequenceList()

In [ ]:
#download data for each specimen_id(do not run this, already downloaded)
def download_data(specimen_id_list):
    try:
        for i in range(0,len(specimen_id_list)):
            ctc.get_ephys_data(specimen_id_list[i])
    except:
        print("Error while downloading")
    return "Files downloaded"



In [ ]:
import numpy as np
def stimulus_response_long_square(stimulus_value,specimen_id_list):
    i=[]
    v=[]
    t=[]
    spec_id=[]
    dataset={}
    for specimen_id in specimen_id_list:
        data_set_dir='/share/quonlab/data/allen/'
        specimen_filename='specimen_'+str(specimen_id)+'/ephys.nwb'
        file_name = data_set_dir+specimen_filename
        data_set = NwbDataSet(file_name)
        sweeps = ctc.get_ephys_sweeps(specimen_id) 
        # groups the sweeps by stimulus 
        sweep_numbers = defaultdict(list)
        for sweep in sweeps:
            sweep_numbers[sweep['stimulus_name']].append(sweep['sweep_number'])

        for sweep_number in sweep_numbers['Long Square']:
            sweep_metadata=data_set.get_sweep_metadata(sweep_number)
            #change the range accordingly 
            if int(sweep_metadata['aibs_stimulus_amplitude_pa']) in range(stimulus_value,stimulus_value+1) and specimen_id not in spec_id:
                sweep_data = data_set.get_sweep(sweep_number)
    #             # sampling rate is in Hz
                sampling_rate = sweep_data['sampling_rate']
                if int(sampling_rate) == 200000:
    #                 samplingRate.append(specimen_id)
                    index_range = sweep_data["index_range"]
                    temp_i = sweep_data["stimulus"][0:index_range[1]+1] # in A
                    temp_v = sweep_data["response"][0:index_range[1]+1] # in V
                    temp_i *= 1e12 # to pA
                    temp_v *= 1e3 # to mV
                    temp_t = np.arange(0, len(temp_v)) * (1.0 / sampling_rate)
                    start_idx=np.where(temp_t==1.01)[0][0]
                    end_idx=np.where(temp_t==2.02)[0][0]
                    temp_t = temp_t[start_idx:end_idx][0::2]
                    temp_i = temp_i[start_idx:end_idx][0::2]
                    #change this to common denominator 
                    temp_v=temp_v[start_idx:end_idx][0::2]
                    i.append(temp_i)
                    v.append(temp_v)
                    t.append(temp_t)
                    dataset[specimen_id]={'i':temp_i,'v':temp_v,'t':temp_t}
                else:
    #                 samp_notEqual.append(sampling_rate)
    #                 specid_notEqual.append(specimen_id)
                    index_range = sweep_data["index_range"]
                    temp_i = sweep_data["stimulus"][0:index_range[1]+1] # in A
                    temp_v = sweep_data["response"][0:index_range[1]+1] # in V
                    temp_i *= 1e12 # to pA
                    temp_v *= 1e3 # to mV
                    temp_t = np.arange(0, len(temp_v)) * (1.0 / sampling_rate)
                      #change this to common denominator 
                    start_idx=np.where(temp_t==1.01)[0][0]
                    end_idx=np.where(temp_t==2.02)[0][0]
                    temp_t = temp_t[start_idx:end_idx][0::5]
                    temp_i = temp_i[start_idx:end_idx][0::5]
                    temp_v=temp_v[start_idx:end_idx][0::5]
                    i.append(temp_i)
                    v.append(temp_v)
                    t.append(temp_t)
                    dataset[specimen_id]={'i':temp_i,'v':temp_v,'t':temp_t}

                spec_id.append(specimen_id)
    print(spec_id)

    data_dict={
        'i':i,
        'v':v,
        't':t,
        'id':spec_id        
    }
    with open ('human_cells_170_stimulus.pkl','wb') as pf:
        pickle.dump(data_dict,pf)
#     with open ('humancell_dataset_170_stimulus_longSq.pkl','wb') as pf:
#         pickle.dump(dataset,pf)

In [ ]:
import numpy as np
def stimulus_response_noise1(stimulus_value,specimen_id_list):
    i=[]
    v=[]
    t=[]
    spec_id=[]
    for specimen_id in specimen_id_list:
        data_set_dir='/share/quonlab/data/allen/'
        specimen_filename='specimen_'+str(specimen_id)+'/ephys.nwb'
        file_name = data_set_dir+specimen_filename
        data_set = NwbDataSet(file_name)
        sweeps = ctc.get_ephys_sweeps(specimen_id) 
        # groups the sweeps by stimulus 
        sweep_numbers = defaultdict(list)
        for sweep in sweeps:
            sweep_numbers[sweep['stimulus_name']].append(sweep['sweep_number'])
            
        if 'Noise 1' in sweep_numbers:
            for sweep_number in sweep_numbers['Noise 1']:
                sweep_metadata=data_set.get_sweep_metadata(sweep_number)
                if int(sweep_metadata['aibs_stimulus_amplitude_pa']) in range(stimulus_value,700) and specimen_id not in spec_id:
                    sweep_data = data_set.get_sweep(sweep_number)
                    sampling_rate = sweep_data['sampling_rate']
                    if int(sampling_rate) == 200000:
                        try:
                            index_range = sweep_data["index_range"]
#                             temp_i = sweep_data["stimulus"][0:index_range[1]+1] # in A
                            temp_v = sweep_data["response"][0:index_range[1]+1] # in V
#                             temp_i *= 1e12 # to pA
                            temp_v *= 1e3 # to mV
                            temp_t = np.arange(0, len(temp_v)) * (1.0 / sampling_rate)
                            start_idx=np.where(temp_t==2.02)[0][0]
                            end_idx=np.where(temp_t==22.01)[0][0]
#                             temp_t = temp_t[start_idx:end_idx][0::4]
#                             temp_i = temp_i[start_idx:end_idx][0::4]
                            temp_v=temp_v[start_idx:end_idx][0::2]
#                             i.append(temp_i)
                            v.append(temp_v)
#                             t.append(temp_t)
                        except:
                            print("error ocurred for",specimen_id)
                    else:
                        index_range = sweep_data["index_range"]
#                         temp_i = sweep_data["stimulus"][0:index_range[1]+1] # in A
                        temp_v = sweep_data["response"][0:index_range[1]+1] # in V
#                         temp_i *= 1e12 # to pA
                        temp_v *= 1e3 # to mV
                        temp_t = np.arange(0, len(temp_v)) * (1.0 / sampling_rate)
                        start_idx=np.where(temp_t==2.02)[0][0]
                        end_idx=np.where(temp_t==22.01)[0][0]
#                         temp_t = temp_t[start_idx:end_idx]
#                         temp_i = temp_i[start_idx:end_idx]
                        temp_v=temp_v[start_idx:end_idx][0::5]
#                         i.append(temp_i)
                        v.append(temp_v)
#                         t.append(temp_t)

                    spec_id.append(specimen_id)
#uncomment if you want to save in pickle format            
#     data_dict={
#         'i':i,
#         'v':v,
#         't':t
#     }
#     with open ('voltage_pick_noise1.pkl','wb') as pf:
#         pickle.dump(data_dict,pf)
    return v

In [ ]:
def read_pickleFile(filename):
    # returns dump pickle file object
    with open (filename,'rb') as pf:
        new_data=pickle.load(pf)
    return new_data



In [ ]:
filename='human_cells_170_stimulus.pkl'
new_data=read_pickleFile(filename)
i=new_data['i']
t=new_data['t']
v=new_data['v']
ids=new_data['id']

In [ ]:
#converting to df
import pandas as pd
import numpy as np
stimulus_df = pd.DataFrame(i[0],columns=["stimulus"])
response_df = pd.DataFrame(v[0],columns=["response"])
time_df = pd.DataFrame(t[0],columns=["time"])
for i_index in range(1,len(i)):
    response_df=response_df.append(pd.DataFrame(v[i_index],columns=["response"]),ignore_index=True)
    stimulus_df=stimulus_df.append(pd.DataFrame(i[i_index],columns=["stimulus"]),ignore_index=True)
    time_df=time_df.append(pd.DataFrame(t[i_index],columns=["time"]),ignore_index=True)



In [ ]:

df=time_df.join(stimulus_df)
df=df.join(response_df)
df




In [1]:
#to extract metadata
import numpy as np
import h5py
def extract_age(specimen_id_list):
    spec_id=[]
    id_age={}
    details={}
    for specimen_id in specimen_id_list:
        print(specimen_id)
        data_set_dir='/share/quonlab/data/allen/'
        specimen_filename='specimen_'+str(specimen_id)+'/ephys.nwb'
        file_name = data_set_dir+specimen_filename
        data=0
        sex=''
        with h5py.File(file_name, 'r') as fi:
            for key in fi.keys():
                group = fi['general']['subject']
                if key == 'general':
#                     print(group)
                    for key in group.keys():
            #             print(key)
                        if key=='age':
                            data = group[key].value
                            print(data)
                        if key=='sex':
                            sex = group[key].value
#                             print(data)    
    
#         ages.append(data) 
        id_age[specimen_id]={'sex':sex,"age":data}
        fi.close()
    return id_age,new_data
       print(key)

id_age,new_data=extract_age(ids)
